In [12]:
import numpy as np
import random
import csv

data = []

# Betöltés
# kódolási séma specifikálása
with open('data.csv','r', encoding='utf-8-sig') as f:
    csvreader = csv.DictReader(f)
    for item in csvreader:
        data.append([ item['DATE'], item['AUTHOR'], item['CONTENT'], item['CLASS'] ])

# A 'data' tömb elemei: ['dátum string', 'szerző', 'komment', 'osztály cimke ('0': nem spam, '1': spam)']
        
# Train/test szétválasztás
split = 0.7
data = np.asarray(data)
perm = np.random.permutation(len(data))

train = data[perm][0:int(len(data)*split)]
test = data[perm][int(len(data)*split):]

print('Train set: ', np.shape(train))
print('Test set: ', np.shape(test))

# Buta osztályozó
def dumb_classify(data):
    threshold = 0.3
    if random.random() > threshold:
        return '1'
    else:
        return '0'


# Használd a 'train' adatokat az osztályozó módszer kidolgozására, a 'test' adatokat kiértékelésére!
# Lehetőleg használj gépi tanulást!
# Dokumentáld az érdekesnek tartott kísérleteket is!

# Példa kiértékelés 'recall' számításával. 
# Kérdés: Milyen egyéb metrikát használnál kiértékelésre és miért? 
sum_positive = 0
found_positive = 0

for datapoint in test:
    if datapoint[-1] == '1':
        sum_positive += 1
        if dumb_classify(datapoint) == '1':
            found_positive += 1
    
print('Recall:', found_positive / sum_positive)

Train set:  (1367, 4)
Test set:  (586, 4)
Recall: 0.6920415224913494


## Előfeldolgozás

In [32]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import EnglishStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import emoji
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

tokenizer = TweetTokenizer()
stemmer = EnglishStemmer()

In [15]:
train_df = pd.DataFrame(train, columns=["date", "author", "text", "spam"])
test_df = pd.DataFrame(test, columns=["date", "author", "text", "spam"])

In [17]:
x = train_df['text'].iloc[2]
tokenizer.tokenize(x.lower())

['rihanna',
 'is',
 'so',
 'beautiful',
 'and',
 'amazing',
 '♥',
 '♥',
 '♥',
 'love',
 'her',
 'so',
 'much',
 '♥',
 '♥',
 '♥',
 'forever',
 'riri',
 'fan',
 '♥',
 '♥',
 '♥',
 '♥',
 '♥',
 '♥',
 '\ufeff']

In [18]:
def process_word(word):
    # szamok helyettesitese
    if re.match(r'\d+', word) is not None:
        return '$'
    # emoji es smiley helyettesitese
    elif re.match(r'(?::|;|=)(?:-)?(?:\)|D|P|\()', word) is not None or word in emoji.UNICODE_EMOJI:
        return 'EMOJI'
    # url linkek helyettesitese
    elif re.match(r'((http|https):\/\/)?([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])?', word) is not None:
        return 'URL'
    # TODO remove stopwords
    else:
        # irasjelek eltavolitasa
        out = re.sub('[^A-Za-z0-9]+', '', word)
        # stemmelés
        return(stemmer.stem(out))
    
def preprocessor(comment_text):
    # \ufeff karakterek eltűntetése
    clean_text = comment_text.replace('\ufeff', '')
    # kisbetűsítés és tokenizálás
    token_list = tokenizer.tokenize(clean_text.lower())
    return [process_word(word) for word in token_list]

In [28]:
print('Egy példa a preprocesszor működéséről')
print(x)
print(preprocessor(x))

Egy példa a preprocesszor működéséről
Rihanna is so beautiful and amazing ♥♥♥♥♥love her so much♥♥♥♥ forever RiRi fan ♥ ♥ ♥ ♥ ♥ ♥ ﻿
['rihanna', 'is', 'so', 'beauti', 'and', 'amaz', 'EMOJI', 'EMOJI', 'EMOJI', 'love', 'her', 'so', 'much', 'EMOJI', 'EMOJI', 'EMOJI', 'forev', 'riri', 'fan', 'EMOJI', 'EMOJI', 'EMOJI', 'EMOJI', 'EMOJI', 'EMOJI']


In [39]:
# vektorizalas eloszor count vektorizatorral
vectorizer = CountVectorizer(lowercase=True,
                             stop_words='english',
                             ngram_range=(1, 1), 
                             analyzer=preprocessor, 
                             max_df=1.0, 
                             min_df=1, 
                             max_features=None)
X_train = vectorizer.fit_transform(train_df['text'])
y_train = train_df['spam']
X_test = vectorizer.transform(test_df['text'])
y_test = test_df['spam']

In [40]:
vectorizer.vocabulary_

{'': 0,
 'sttuupid': 2058,
 'repli': 1780,
 'buck': 289,
 'mmlp': 1392,
 'feedback': 766,
 'compani': 417,
 'amiabl': 97,
 'meat': 1337,
 'wahoo': 2330,
 'useless': 2275,
 'king': 1186,
 'oper': 1546,
 'ittttttttttttt': 1126,
 'famili': 739,
 'hop': 1017,
 'ociramma': 1516,
 'dant': 515,
 'singer': 1933,
 'adhoc': 40,
 'gonna': 899,
 'wat': 2349,
 'touch': 2197,
 'lol': 1264,
 'drone': 610,
 'strateg': 2051,
 'trash': 2211,
 'stereotyp': 2042,
 'which': 2378,
 'upcom': 2265,
 'war': 2340,
 'aid': 66,
 'heard': 978,
 'axi': 174,
 'martyr': 1327,
 'project': 1690,
 'order': 1552,
 'troll': 2219,
 'plizz': 1639,
 'write': 2432,
 'this': 2163,
 'soccer': 1968,
 'partyman': 1575,
 'core': 455,
 'n': 1443,
 'hart': 963,
 'hit': 1007,
 'lip': 1255,
 'bonus': 257,
 'acquir': 29,
 'dis': 566,
 'am': 89,
 'gangnamstyl': 867,
 'aunt': 161,
 'comentar': 410,
 'divin': 576,
 'imag': 1064,
 'market': 1323,
 'monster': 1404,
 'fruiti': 840,
 'littl': 1257,
 'href': 1029,
 'insidi': 1093,
 'therealchr

## Multinomial Naive Bayes classifier

A spam detektálás egy klasszifikációs feladat, amelyet az irodolom szerint a Naív Bayes osztályozók kifejezetten jól képesek kezelni. A Naív Bayes-féle osztályozók továbbá kellően robosztusak, gyorsak és kevés tanulóadattal is jól működnek, így baseline modellként érdemes ezzel kezdeni.

Mivel a nyelvi adatok feldolgozásához gyakorisági alapú vektorizálót használtam, így a naív Bayes osztályozók multinomiális változatát választottam.

In [43]:
multi_nb = MultinomialNB()
multi_nb.fit(X_train, y_train)
print('Recall a tanulóadatokon')
print(classification_report(y_train, multi_nb.predict(X_train)))
print('Recall a tesztadatokon')
print(classification_report(y_test, multi_nb.predict(X_test)))

Recall a tanulóadatokon
             precision    recall  f1-score   support

          0       0.96      0.95      0.95       653
          1       0.95      0.96      0.96       714

avg / total       0.96      0.96      0.96      1367

Recall a tesztadatokon
             precision    recall  f1-score   support

          0       0.95      0.84      0.89       297
          1       0.85      0.95      0.90       289

avg / total       0.90      0.90      0.90       586



In [44]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, multi_nb.predict(X_test))

array([[250,  47],
       [ 14, 275]])

In [42]:
# todo
# SVM, tfidf, stop words, feature engineering, CV, parameter tuning

## Lehetséges változók

1. Url link; kulcsszabak *check out, visit, subscribe, follow me, money*
2. Nagybetűk, írásjelek pl. felkiáltójelek aránya
3. Felhasználó (csak 21 komment van ugyanazon felhasználótól a spamok között) ``np.unique(spams[:, 1])``
4. Posztolás ideje

## Modellek építése

## Lehetséges problémak

### Osztályok közötti aránytalanság

In [ ]:
spams = train[np.where(train == '1')[0]]
len(train)/len(spams)

Majdnem 2-szer annyi a negatív példa, mint pozitív.

In [ ]:
# egyedi felhasznalonevek szama
len(spams) - len(np.unique(spams[:, 1]))